# CosMx Data Preprocessing Pipeline

This notebook processes NanoString CosMx spatial transcriptomics data for the Pancreas dataset.


## 1. Extract DAPI Images from CellComposite

### CellComposite Image Structure

Each FOV has a CellComposite image displaying ImmunoFluorescence intensity from IHC markers selected for the SMI experiment.

**Channels:**
- **Green**: PanCK
- **Red**: CD45/GFAP
- **Yellow**: CD68/CK8_18
- **Blue**: DAPI (nuclear staining)

We extract the blue channel (DAPI) from CellComposite images and save them as separate TIFF files.


In [ ]:
# Import required libraries
import os
import sys
from PIL import Image
import numpy as np
import tifffile
import pandas as pd
import matplotlib.pyplot as plt
import natsort
import tempfile
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


In [ ]:
# Define data paths
NanoString_CosMx_Human_Pancreas_path = r"D:/segmentation_datasets/NanoString_CosMx_Human_Pancreas/Pancreas-CosMx-WTx-FlatFiles/"

CellComposite_path = os.path.join(NanoString_CosMx_Human_Pancreas_path, "CellComposite")
DAPI_path = os.path.join(NanoString_CosMx_Human_Pancreas_path, "DAPI")

# Create DAPI output directory if it doesn't exist
os.makedirs(DAPI_path, exist_ok=True)

print(f"CellComposite path: {CellComposite_path}")
print(f"DAPI output path: {DAPI_path}")


In [ ]:
# Extract DAPI channel from CellComposite images
for file_name in os.listdir(CellComposite_path):
    # Process only CellComposite .jpg files
    if "CellComposite" in file_name and file_name.lower().endswith(".jpg"):
        file_path = os.path.join(CellComposite_path, file_name)
        print(f"Processing file: {file_path}")
        
        # Load image
        img = Image.open(file_path)
        img_array = np.array(img)
        
        # Check if image is RGB
        if img_array.ndim == 3 and img_array.shape[2] >= 3:
            # Extract blue channel (DAPI channel, index 2)
            dapi_channel = img_array[:, :, 2]
            
            # Extract FOV number from filename (e.g., "CellComposite_F051.jpg" -> "F051")
            parts = file_name.split("_")
            if len(parts) >= 2:
                f_part = parts[-1]  # "F051.jpg"
                f_num = os.path.splitext(f_part)[0]  # "F051"
                new_name = f"DAPI_{f_num}.tif"
            else:
                new_name = "DAPI_" + os.path.splitext(file_name)[0] + ".tif"
            
            out_file = os.path.join(DAPI_path, new_name)
            
            # Save DAPI channel as TIFF (lossless)
            tifffile.imwrite(out_file, dapi_channel)
            print(f"Saved DAPI image: {out_file}")
        else:
            print(f"File {file_name} is not a standard RGB image, skipping.")

print("\nDAPI extraction complete!")


## 2. Generate Gene Expression Maps

Process the Pancreas_tx_file.csv to generate gene expression maps for each FOV.
Each pixel in the gene map represents the transcript count at that location.


In [ ]:
# Load transcript data
Pancreas_tx_file = os.path.join(NanoString_CosMx_Human_Pancreas_path, "Pancreas_tx_file.csv")
df_tx = pd.read_csv(Pancreas_tx_file)

print(f"Loaded transcript file: {Pancreas_tx_file}")
print(f"Total transcripts: {len(df_tx)}")


C:\Users\lyh\AppData\Local\Temp\ipykernel_48164\1332432986.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tx = pd.read_csv(Pancreas_tx_file)


In [ ]:
# Preview transcript data
df_tx.head()


,fov,cell_ID,cell,x_local_px,y_local_px,x_global_px,y_global_px,z,target,CellComp
0,51,0,c_1_51_0,86,23,52525.035818,41902.406046,2,TRAPPC8,NaN
1,51,0,c_1_51_0,85,38,52524.036799,41887.408866,2,APOL1,NaN
2,51,0,c_1_51_0,85,38,52524.036799,41887.408866,3,APOL1,NaN
3,51,0,c_1_51_0,95,45,52534.038883,41880.407804,1,NCF4,NaN
4,51,0,c_1_51_0,83,56,52522.038761,41869.406700,3,MLN,NaN


In [ ]:
# Analyze FOV 51 data
fov51_df = df_tx[df_tx['fov'] == 51]

# Count unique genes
unique_genes = fov51_df['target'].unique()
print(f"FOV 51 contains {len(unique_genes)} unique genes")
print("Genes:", unique_genes)

# Get coordinate ranges
x_min = fov51_df['x_local_px'].min() 
x_max = fov51_df['x_local_px'].max()
y_min = fov51_df['y_local_px'].min()
y_max = fov51_df['y_local_px'].max()

print(f"\nFOV 51 has {len(fov51_df)} transcripts")
print(f"x_local_px range: ({x_min}, {x_max})")
print(f"y_local_px range: ({y_min}, {y_max})")


### Select Top 500 Most Abundant Genes

Filter transcripts to keep only the top 500 most highly expressed genes.
This reduces computational requirements while maintaining biological signal.


In [ ]:
# Filter FOV 51 data
fov51_df = df_tx[df_tx['fov'] == 51]

print("======> FOV 51 Original Data Statistics")
# Count unique genes
unique_genes = fov51_df['target'].unique()
print(f"FOV 51 contains {len(unique_genes)} unique genes")
print("Genes:", unique_genes)

# Calculate coordinate ranges for original data
x_min_full = fov51_df['x_local_px'].min() 
x_max_full = fov51_df['x_local_px'].max()
y_min_full = fov51_df['y_local_px'].min()
y_max_full = fov51_df['y_local_px'].max()

map_width_full = x_max_full - x_min_full + 1
map_height_full = y_max_full - y_min_full + 1

print(f"Original x_local_px range: ({x_min_full}, {x_max_full})")
print(f"Original y_local_px range: ({y_min_full}, {y_max_full})")
print(f"Original gene map size: ({map_height_full}, {map_width_full})\n")

# Count transcripts per gene and select top 500
gene_counts = fov51_df['target'].value_counts()
top500_genes = gene_counts.head(500).index  # Returns all genes if less than 500

print(f"Top {len(top500_genes)} most expressed genes and their transcript counts:")
print(gene_counts.loc[top500_genes])

# Filter data to keep only top 500 genes
fov51_top500_df = fov51_df[fov51_df['target'].isin(top500_genes)]

# Calculate coordinate ranges for filtered data
x_min_top500 = fov51_top500_df['x_local_px'].min() 
x_max_top500 = fov51_top500_df['x_local_px'].max()
y_min_top500 = fov51_top500_df['y_local_px'].min()
y_max_top500 = fov51_top500_df['y_local_px'].max()

map_width_top500 = x_max_top500 - x_min_top500 + 1
map_height_top500 = y_max_top500 - y_min_top500 + 1

print("\n======> Filtered Data Statistics (Top 500 Genes)")
# Count unique genes in filtered data
unique_genes_top500 = fov51_top500_df['target'].unique()
print(f"Filtered FOV 51 contains {len(unique_genes_top500)} unique genes")
print("Genes:", unique_genes_top500)

print(f"Filtered x_local_px range: ({x_min_top500}, {x_max_top500})")
print(f"Filtered y_local_px range: ({y_min_top500}, {y_max_top500})")
print(f"Filtered gene map size: ({map_height_top500}, {map_width_top500})")


In [ ]:
# Save filtered gene data and statistics
print(f"Generating Gene Map for top 500 most abundant genes")
print(f"Number of genes: {len(top500_genes)}")
print(f"Number of transcripts: {len(fov51_top500_df)}")

# Create output directory for top 500 genes
top500_out_dir = os.path.join("D:/paper/newmodel/data/NanoString_CosMx_Human_Pancreas/fov51", "top500")
os.makedirs(top500_out_dir, exist_ok=True)

# Save gene list
with open(os.path.join(top500_out_dir, 'top500_gene_names.txt'), 'w') as f:
    for gene in top500_genes:
        f.write(gene + '\n')

# Save filtered transcript data
fov51_top500_df.to_csv(os.path.join(top500_out_dir, 'top500_transcripts.csv'), index=False)

print(f"Saved gene list to: {os.path.join(top500_out_dir, 'top500_gene_names.txt')}")
print(f"Saved transcript data to: {os.path.join(top500_out_dir, 'top500_transcripts.csv')}")


In [ ]:
# Generate multi-channel gene expression map for top 500 genes
print("Generating Gene Map for top 500 most abundant genes...")

# Create temporary directory for individual gene maps
temp_dir_top500 = tempfile.mkdtemp(dir="D:/paper/newmodel/temp_dir")
os.makedirs(temp_dir_top500, exist_ok=True)
print(f"Created temporary directory: {temp_dir_top500}")

# Function to process a chunk of genes
def process_gene_chunk_top500(gene_chunk):
    for gene in gene_chunk:
        # Filter transcripts for current gene
        df_gene = fov51_top500_df[fov51_top500_df['target'] == gene]
        # Create blank image
        gene_map = np.zeros((map_height_full, map_width_full), dtype=np.uint8)
        
        # Mark transcript locations on the image
        for _, transcript in df_gene.iterrows():
            x = int(np.round(transcript['x_local_px'] - x_min_full))
            y = int(np.round(transcript['y_local_px'] - y_min_full))
            
            # Ensure coordinates are within valid range
            if 0 <= x < map_width_full and 0 <= y < map_height_full:
                gene_map[y, x] += 1  # Increment count at transcript location
        
        # Save individual gene expression map
        tifffile.imwrite(os.path.join(temp_dir_top500, f"{gene}.tif"), gene_map, photometric='minisblack')

# Parallel processing of all high-expression genes
print("Generating individual gene expression maps in parallel...")
n_workers = min(32, os.cpu_count() * 2) 
gene_chunks_top500 = np.array_split(top500_genes, n_workers)

with ThreadPoolExecutor(max_workers=n_workers) as executor:
    list(tqdm(executor.map(process_gene_chunk_top500, gene_chunks_top500), 
              total=len(gene_chunks_top500), 
              desc="Processing gene chunks"))

# Merge all gene expression maps into a multi-channel image
print("Merging all gene expression maps...")
gene_map_all_top500 = np.zeros((map_height_full, map_width_full, len(top500_genes)), dtype=np.uint8)

for i, gene in enumerate(tqdm(top500_genes, desc="Merging channels")):
    gene_map_all_top500[:, :, i] = tifffile.imread(os.path.join(temp_dir_top500, f"{gene}.tif"))

# Save multi-channel gene expression map
gene_map_path_top500 = os.path.join(top500_out_dir, "top500_gene_map.tif")
tifffile.imwrite(gene_map_path_top500, gene_map_all_top500)
print(f"Successfully saved gene expression map: {gene_map_path_top500}")

print("\n--- Top 500 Gene Selection and Gene Map Generation Complete ---")
print(f"Results saved in: {top500_out_dir}")


In [ ]:
# Generate gene name to index mapping file
gene_index_df_top500 = pd.DataFrame({
    'gene': top500_genes,
    'index': range(len(top500_genes))
})
gene_index_df_top500.to_csv(os.path.join(top500_out_dir, 'gene_index_map.csv'), index=False)
print(f"Saved gene index mapping: {os.path.join(top500_out_dir, 'gene_index_map.csv')}")

# Save gene expression statistics
gene_counts.loc[top500_genes].to_csv(os.path.join(top500_out_dir, 'gene_expression_counts.csv'))
print(f"Saved gene expression statistics: {os.path.join(top500_out_dir, 'gene_expression_counts.csv')}")


## 3. Cell Segmentation with Cellpose

Segment nuclei from DAPI images using the Cellpose model.


In [ ]:
# Define DAPI image path for segmentation
dapi_path = "D:/segmentation_datasets/ucs_data/NanoString_CosMx_Human_Pancreas/Pancreas-CosMx-WTx-FlatFiles/DAPI/DAPI_F051.tif"

In [ ]:
# Cellpose segmentation for CosMx DAPI images
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import save_masks
from tqdm import tqdm

# DAPI image path
target_fov_dapi_filename = "DAPI_F051.tif"
dapi_base_path = r"D:/segmentation_datasets/ucs_data/NanoString_CosMx_Human_Pancreas/Pancreas-CosMx-WTx-FlatFiles/DAPI/"
dapi_path = os.path.join(dapi_base_path, target_fov_dapi_filename)

# Output directory for segmentation results
target_fov = 51
output_dir_base = r"D:/segmentation_datasets/ucs_data/NanoString_CosMx_Human_Pancreas/Pancreas-CosMx-WTx-FlatFiles/Gene_map/"
output_dir = os.path.join(output_dir_base, f"fov{target_fov}")
os.makedirs(output_dir, exist_ok=True)

# Load DAPI image
print(f"Loading DAPI image: {dapi_path}")
if not os.path.exists(dapi_path):
    print(f"Error: DAPI image file not found: {dapi_path}")
else:
    dapi_img = tifffile.imread(dapi_path)

    # Visualize input image (optional)
    plt.figure(figsize=(10, 10))
    plt.imshow(dapi_img, cmap='gray')
    plt.title(f"DAPI Image (FOV {target_fov})")
    plt.colorbar()
    plt.close()

    # Image normalization
    print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")
    if dapi_img.max() > 0:
        if dapi_img.max() > 255 and dapi_img.dtype != np.uint16:
            print("Normalizing image to 0-255 (uint8)")
            dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
        elif dapi_img.dtype == np.uint16:
            print("Image is uint16 type, keeping original format")
            dapi_img_norm = dapi_img.copy()
        else:
            print("Image is already in suitable intensity range")
            dapi_img_norm = dapi_img.astype(np.uint8)
    else:
        print("Warning: Image max value is 0, might be empty image")
        dapi_img_norm = dapi_img.astype(np.uint8)

    # Initialize Cellpose model
    print("Initializing Cellpose model (model_type='nuclei')...")
    model = models.Cellpose(
        gpu=True,
        model_type='nuclei'
    )

    # Segmentation parameters optimized for CosMx DAPI images
    diameter = None  # Auto-estimate or specify based on your data
    flow_threshold = 0.4
    cellprob_threshold = 0.0

    print(f"Running Cellpose segmentation with parameters:")
    print(f"- diameter: {diameter} (auto-estimate if None)")
    print(f"- flow_threshold: {flow_threshold}")
    print(f"- cellprob_threshold: {cellprob_threshold}")

    # Run Cellpose segmentation
    masks, flows, styles, diams_estimated = model.eval(
        dapi_img_norm,
        diameter=diameter,
        channels=[0, 0],
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        do_3D=False
    )
    
    if diameter is None or diameter == 0:
        print(f"Cellpose estimated nucleus diameter: {diams_estimated}")

    print(f"Segmentation complete. Found {masks.max()} cells.")

    # Save mask image
    mask_filename = f"cellpose_masks_fov{target_fov}.tif"
    mask_path = os.path.join(output_dir, mask_filename)
    tifffile.imwrite(mask_path, masks.astype(np.uint16))
    print(f"Mask file saved: {mask_path}")

    # Save Cellpose visualization output
    segmentation_vis_filename = f"segmentation_vis_fov{target_fov}.png"
    save_masks(dapi_img_norm, masks, flows, os.path.join(output_dir, segmentation_vis_filename))
    print(f"Segmentation visualization saved: {os.path.join(output_dir, segmentation_vis_filename)}")

    # Create and save overlay visualization
    plt.figure(figsize=(15, 15))
    plt.imshow(dapi_img_norm, cmap='gray')
    plt.imshow(masks > 0, alpha=0.3, cmap='viridis')
    plt.title(f"DAPI (FOV {target_fov}) with Cell Segmentation Overlay")
    overlay_filename = f"dapi_segmentation_overlay_fov{target_fov}.png"
    plt.close()

    # Generate and save boundary image
    from skimage.segmentation import find_boundaries
    boundaries = find_boundaries(masks, mode='outer', background=0)
    plt.figure(figsize=(15, 15))
    plt.imshow(dapi_img_norm, cmap='gray')
    plt.imshow(boundaries, alpha=0.8, cmap='Reds_r')
    plt.title(f"DAPI (FOV {target_fov}) with Cell Boundaries")
    boundaries_filename = f"cell_boundaries_fov{target_fov}.png"
    plt.close()

    print(f"All results saved to: {output_dir}")


## 4. Cell Segmentation with Cellpose-SAM

Segment nuclei using the Cellpose-SAM model, which combines Cellpose and SAM for improved segmentation.


In [ ]:
# Import libraries for Cellpose-SAM
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt
from natsort import natsorted

# Set up logging
io.logger_setup()

# Initialize Cellpose-SAM model
model = models.CellposeModel(gpu=True)


In [ ]:
# Cellpose-SAM segmentation for CosMx DAPI images
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()

# DAPI image path
target_fov_dapi_filename = "DAPI_F051.tif"
dapi_base_path = r"D:/segmentation_datasets/ucs_data/NanoString_CosMx_Human_Pancreas/Pancreas-CosMx-WTx-FlatFiles/DAPI/"
dapi_path = os.path.join(dapi_base_path, target_fov_dapi_filename)

# Output directory for segmentation results
target_fov = 51
output_dir_base = r"D:/paper/newmodel/data/NanoString_CosMx_Human_Pancreas/fov51/"
output_dir = os.path.join(output_dir_base, "cellpose_sam_segmentation")
os.makedirs(output_dir, exist_ok=True)

# Load DAPI image
print(f"Loading DAPI image: {dapi_path}")
if not os.path.exists(dapi_path):
    print(f"Error: DAPI image file not found: {dapi_path}")
    exit()

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - ensure correct format
if len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to new Cellpose format
# For DAPI single-channel image, we use the first channel
first_channel = '0'   # Use the first channel (DAPI)
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 32          # Batch size

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

# Save segmentation results
mask_filename = f"cellpose_sam_masks_fov{target_fov}.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint16))
print(f"Mask file saved: {mask_path}")

# Generate visualization using new version method
print("Generating visualization results...")
fig = plt.figure(figsize=(12, 5))
plot.show_segmentation(fig, img_selected_channels, masks, flows[0])
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"cellpose_sam_segmentation_fov{target_fov}.png"), 
            dpi=300, bbox_inches='tight')
plt.close()

# Additional visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original image
if len(img_selected_channels.shape) == 3:
    display_img = img_selected_channels[:, :, 0]  # Display first channel
else:
    display_img = img_selected_channels

axes[0].imshow(display_img, cmap='gray')
axes[0].set_title('Original DAPI Image')
axes[0].axis('off')

# Segmentation masks
axes[1].imshow(masks, cmap='viridis')
axes[1].set_title(f'Cellpose-SAM Masks\n({masks.max()} cells)')
axes[1].axis('off')

# Overlay display
axes[2].imshow(display_img, cmap='gray')
axes[2].imshow(masks > 0, alpha=0.3, cmap='viridis')
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"cellpose_sam_results_fov{target_fov}.png"), 
            dpi=300, bbox_inches='tight')
plt.close()

# Save boundary image
from skimage.segmentation import find_boundaries
boundaries = find_boundaries(masks, mode='outer', background=0)

plt.figure(figsize=(12, 12))
plt.imshow(display_img, cmap='gray')
plt.imshow(boundaries, alpha=0.8, cmap='Reds')
plt.title(f'Cell Boundaries (FOV {target_fov}) - Cellpose-SAM')
plt.axis('off')
plt.savefig(os.path.join(output_dir, f"cellpose_sam_boundaries_fov{target_fov}.png"), 
            dpi=300, bbox_inches='tight')
plt.close()

# Save statistics
import json
stats = {
    'model': 'cellpose-sam (cpsam)',
    'num_cells': int(masks.max()),
    'image_shape': list(dapi_img.shape),
    'processed_image_shape': list(img_selected_channels.shape),
    'selected_channels': selected_channels,
    'parameters': {
        'flow_threshold': flow_threshold,
        'cellprob_threshold': cellprob_threshold,
        'tile_norm_blocksize': tile_norm_blocksize,
        'batch_size': batch_size
    }
}

with open(os.path.join(output_dir, f"cellpose_sam_stats_fov{target_fov}.json"), 'w') as f:
    json.dump(stats, f, indent=2)

print(f"\n=== Cellpose-SAM Segmentation Complete ===")
print(f"Results saved to: {output_dir}")
print(f"Number of cells detected: {masks.max()}")
print(f"Model used: cpsam (Cellpose-SAM)")
print(f"Image shape: {img_selected_channels.shape}")

print("\nSegmentation task complete!")
